In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
class Transformer(nn.Module):
    def __init__(self, encoder, decoder):
        super(Transformer, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        
    def forward(self, input_sentence, some_sentence):
        context         = self.encoder(input_sentence)
        output_sentence = self.decoder(some_sentence, context)
        return output_sentence

In [3]:
import copy

class Encoder(nn.Module):
    def __init__(self, encoder_layer, n_layer):
        super(Encoder, self).__init__()
        self.layers = []
        for i in range(n_layer):
            self.layers.append(copy.deepcopy(encoder_layer))
            
    def forward(self, x):
        out = x
        for layer in self.layers:
            out = layer(out)
        return out

In [4]:
class EncoderLayer(nn.Module):
    def __init__(self, multi_head_attention_layer, position_wise_feed_forward_layer):
        super(EncoderLayer, self).__init__()
        self.multi_head_attention_layer = multi_head_attention_layer
        self.position_wise_feed_forward_layer = position_wise_feed_forward_layer
        
    def forward(self, x):
        out = self.multi_head_attention_layer(x)
        out = self.position_wise_feed_forward_layer(out)
        return out

### self-attention
self-Attention 에서의 계산하는 대상은 Query 가 주어졌을 때, 다른 token 에 대한 관계다. 먼저 Query, Key, Value 3가지 input 을 받게 된다.

1. Query: 현재 시점의 token을 의미
2. Key: attention을 구하고자 하는 대상 token을 의미
3. Value: attention을 구하고자 하는 대상 token을 의미 (Key와 동일한 token)

Query, Key, Value 는 input으로 들어오는 token embedding vector를 fully connected layer에 넣어 세 vector를 만들어낸다. ([n, d_embed] -> [n, dk])

수식은 이하와 같이 계산된다. 

$$
\text { Query's Attention }(Q, K, V)=\operatorname{softmax}\left(\frac{Q K^{T}}{\sqrt{d_{k}}}\right) V
$$

Q[1, dk] x K.T[dk, n] = Attention Score [1, n]
즉, Attention score 는 Query 가 각각의 key 와의 유사도를 나타내다. (내적 = 유사도)

이렇게 구한 attention score 를 value 에 곱해주면 attention 결과를 계산할 수 있다. 

AS[1, n] x V[n, dk] = Query's attention [1, dk]

1개의 token 을 n개로 확장하려면 1 을 n 으로 바꿔주기만 하면 된다.

In [7]:
def calculate_attention(self, query, key, value, mask):
    # query, key, value's shape: (n_batch, seq_len, d_k)
    d_k = key.size(-1)
    attention_score = torch.einsum('bik, bjk -> bij', query, key) / torch.sqrt(d_k)
    if mask is not None:
        attention_score.masked_fill_(mask == 0, -1e9)
        # attention_score = attention_score.masked_fill(mask == 0, -1e9) #!# almost pseudo-code..
    attention_prob = F.softmax(attention_score, dim = -1)
    return torch.einsum('bik, bkj -> bij', attention_prob, value)

실제 논문에서는 multi-head attention 모델이 사용되는데, self-attention 을 병렬적으로 여러개 수행하는 것이다. 이를 통해 덜 중요한 attention 까지 포함할 수 있는 attention 을 얻을 수 있도록 돕는다.

h = 8 이라면, query, key, value's shape: (n_batch, seq_len, d_k * h) 가 된다. 이제 d_k * h 를 d_model 이라고 부른다. 

In [ ]:
class MultiHeadAttentionLayer(nn.Module):
    def __init__(self, d_model, h, qkv_fc_layer, fc_layer):
        # qkv_fc_layer's shape: (n_batch, d_embed, d_model)
        # fc_layer's shape: (n_batch, d_model, d_embed)
        super(MultiHeadAttentionLayer, self).__init__()
        self.d_model = d_model
        self.h = h
        self.query_fc_layer = copy.deepcopy(qkv_fc_layer)
        self.key_fc_layer = copy.deepcopy(qkv_fc_layer)
        self.value_fc_layer = copy.deepcopy(qkv_fc_layer)
        self.fc_layer = fc_layer
      
    def forward(self, query, key, value, mask = None):
        # query, key, value's shape: (n_batch, seq_len, d_embed)
        # mask's shape: (n_batch, seq_len, seq_len)
        n_batch = query.shape[0]
        
        def transform(x, fc_layer): 
            # x : (n_batch, seq_len, d_model)
            out = fc_layer(x)
            out = out.view()